# Suicide Rates Overview

![](https://medicaldialogues.in/h-upload/2020/07/21/132022-suicide.webp)

**This kernel is made to check the relationship between suicide and many other factors.**


1. [Data](#1)
2. [Data Cleaning](#2)
3. [Relationship with suicide](#3)
4. [Conclusion](#4)
5. [Reference](#5)

In [ ]:
# for some basic operations
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns

# re(=regular expression operations) for changing string to float
import re

<a id=1> </a>
# **Data:**
There is only one file provided in the dataset, as described below:

1. **master.csv** : country, year, sex, age group, count of suicides, population, suicide rate, country-year composite key, HDI for year, gdpforyear, gdppercapita, generation (based on age grouping average).

In [ ]:
# reading the data for Suicide Rates

df = pd.read_csv('../input/suicide-rates-overview-1985-to-2016/master.csv')

# checking the head of the data

df.head(10)

In [ ]:
# show the brief information about df DataFrame

df.info()

<a id=2> </a>
# Data Cleaning

In [ ]:
# check is there any null object in it

df.isnull().sum()

In [ ]:
# delete 'HDI for year' column to remove null object

df = df.drop('HDI for year', axis=1)

In [ ]:
# change the name of the columns for easy usage

df=df.rename(columns={'country':'Country','year':'Year','sex':'Gender','age':'Age','population':'Population','suicides_no':'NumOfSuicides', 'suicides/100k pop':'SuicidesPer100k',
                          'country-year':'CountryYear',' gdp_for_year ($) ':'GdpForYear','gdp_per_capita ($)':'GdpPerCapital','generation':'Generation'})

In [ ]:
# sort the age
df['Age'] = df['Age'].replace({'5-14 years': '1', '15-24 years': '2', '25-34 years': '3', '35-54 years': '4', '55-74 years': '5', '75+ years': '6'})

# change data type str to int for GdpForYear
df['GdpForYear'] = df['GdpForYear'].apply(lambda x : re.sub("[^\d\.]", "", x))
df['GdpForYear'] = df['GdpForYear'].astype('double')

In [ ]:
# check after finishing data cleaning
df.head()

# **Function for Visualization**

**By defining function, it will help to avoid writing same code. This function set x&y axis, name for each axis and title and the size of graph.**

In [ ]:
def visualization(Title = None, xlabel = None, ylabel= None, xTitle= None):
    fig = plt.figure()
    plt.style.use(['fivethirtyeight'])
    axes = fig.add_axes([0.3, 0.1, 1, 0.8])
    if Title == 'Year & Suicides':
        axes.plot(xlabel, ylabel, color = 'navy', linewidth=3, ls='--')
    elif Title == 'Gender & Suicides':
        axes.bar(xlabel, ylabel, color = ['crimson', 'navy'], linewidth=3)
    else:
        axes.bar(xlabel, ylabel, color = 'navy', linewidth=3)
    axes.set_xlabel(xTitle, fontsize=12)
    axes.set_ylabel('SucidiesPer100k', fontsize=12)
    axes.set_title(Title, fontsize=15)
    if Title == 'Age & Suicides':
        plt.xticks(fontsize=10.5)
        axes.set_xticklabels(('5-14 years', '15-24 years', '25-34 years', '35-54 years', '55-74 years', '75+ years'))
    if Title == 'Gender & Suicides':
        plt.xticks(fontsize=15)
    return axes


<a id=3> </a>
# **Show relationship with Suicide**

**I will use 5 columns to show relationship with Suicide(NumOfSuicides, SucidiesPer100k)**

### - Year
### - Age
### - GdpForYear
### - Generation
### - Gender

## 1. Year

In [ ]:
# make new DataFrame call 'df_year', which contains only 'Year', 'NumOfSuicides', 'Population', 'SucidiesPer100k'

df_year = df.drop(['Country', 'Gender', 'Age', 'CountryYear', 'GdpForYear', 'GdpPerCapital', 'Generation'], axis=1)

# group data based on Year

year_group = df_year.groupby(['Year'], as_index=False).sum()


# reset the SucidiesPer100k
year_group['SucidiesPer100k'] = (year_group['NumOfSuicides'] / year_group['Population']) * 100000

year_group.head()

In [ ]:
year_visualization = visualization(Title= 'Year & Suicides',xlabel=year_group['Year'], ylabel=year_group['SucidiesPer100k'], xTitle='Year')

## 2. Age

In [ ]:
# make new DataFrame call 'df_year', which contains only 'Age', 'NumOfSuicides', 'Population', 'SucidiesPer100k'

df_age = df.drop(['Country', 'Gender', 'Year', 'CountryYear', 'GdpForYear', 'GdpPerCapital', 'Generation'], axis=1)

# group data based on Age

age_group = df_age.groupby(['Age'], as_index=False).sum()


# reset the SucidiesPer100k
age_group['SuicidesPer100k'] = (age_group['NumOfSuicides'] / age_group['Population']) * 100000

age_group.head()

In [ ]:
age_visualization = visualization(Title= 'Age & Suicides',xlabel=age_group['Age'], ylabel=age_group['SuicidesPer100k'], xTitle='Age')

## 3. GdpForYear

In [ ]:
# make new DataFrame call 'df_year', which contains only 'GdpForYear', 'NumOfSuicides', 'Population', 'SucidiesPer100k'

df_GDP = df.drop(['Country', 'Gender', 'Age', 'CountryYear', 'Year', 'GdpPerCapital', 'Generation'], axis=1)

df_GDP.info()

In [ ]:
print("The number of unique value in GdpForYear: ",len(df_GDP['GdpForYear'].unique()))
# check minimum & maximum value in GdpForYear
print("Minimum value in GdpForYear: ", df_GDP['GdpForYear'].min())
print("Maximum value in GdpForYear: ", df_GDP['GdpForYear'].max())

In [ ]:
# since the number of GdpForYear is to large, I will make a new column call 'GdpForYearPer$1M', which divides data with 1,000,000(=$1M)
df_GDP['GdpForYearPer$1M'] = df_GDP['GdpForYear'].apply(lambda x : x /1000000)

# sort the data in GdpForYearPer$1M
df_GDP = df_GDP.sort_values(by=['GdpForYearPer$1M'])

# set the range of GdpForYearPer$1M

range_GDP = {'$0 ~ $100M': np.arange(0,100), '$100M ~ $1B': np.arange(100.0000001,1000), '$1B ~ $10B': np.arange(1000.000001, 10000), '$10B ~ $100B': np.arange(10000.000001, 100000),
            '$100B ~ $1T': np.arange(100000.000001, 1000000), '$1T ~': np.arange(1000000.000001, 20000000)}

# generate "cuts" (bins) and associated labels from `range_GDP`.    

cut_data = [(np.min(v), k) for k, v in range_GDP.items()]
bins, labels = zip(*cut_data)

# bins required to have one more value than labels
bins = list(bins) + [np.inf]
df_GDP['GdpForYearPer$1M'] = pd.cut(df_GDP['GdpForYearPer$1M'], bins=bins, labels=labels)

# drop GdpForYear column
df_GDP = df_GDP.drop('GdpForYear', axis=1)

# check the data
df_GDP.head()

In [ ]:
# group data based on GdpForYear

GDP_group = df_GDP.groupby(['GdpForYearPer$1M'], as_index=False).sum()

# reset the SucidiesPer100k
GDP_group['SuicidesPer100k'] = (GDP_group['NumOfSuicides'] / GDP_group['Population']) * 100000

GDP_group.head()

In [ ]:
GDP_visualization = visualization(Title= 'GDP & Suicides',xlabel=GDP_group['GdpForYearPer$1M'], ylabel=GDP_group['SuicidesPer100k'], xTitle='GdpForYearPer$1M')

## 4. Generation

In [ ]:
# make new DataFrame call 'df_Gene', which contains only 'Generation', 'NumOfSuicides', 'Population', 'SucidiesPer100k'

df_Gene = df.drop(['Country', 'Gender', 'Age', 'CountryYear', 'Year', 'GdpPerCapital', 'GdpForYear'], axis=1)

# group data based on Generation

Gene_group = df_Gene.groupby(['Generation'], as_index=False).sum()

# reset the SucidiesPer100k
Gene_group['SuicidesPer100k'] = (Gene_group['NumOfSuicides'] / Gene_group['Population']) * 100000


Gene_group.head()

Sort data by following years order

1. G.I. Generation = 1901 ~ 1945
2. Boomers = 1946 ~ 1964
3. Generation X = 1965 ~ 1976
4. Millenials = 1977 ~ 1995
5. Generation Z = 1996 ~ TBD

In [ ]:
# put Generation column to index for reordering

Gene_group = Gene_group.set_index('Generation')

# make order list for Generation
reorderlist = ['G.I. Generation', 'Boomers', 'Generation X', 'Millenials', 'Generation Z']

# reorder the Generation
Gene_group = Gene_group.reindex(reorderlist)

# put Generation index into column
Gene_group = Gene_group.reset_index()

# check the DataFrame
Gene_group.head()

In [ ]:
Gene_visualization = visualization(Title= 'Generation & Suicides',xlabel=Gene_group['Generation'], ylabel=Gene_group['SuicidesPer100k'], xTitle='Generation')

## 5. Gender

In [ ]:
# make new DataFrame call 'df_gender', which contains only 'Gender', 'NumOfSuicides', 'Population', 'SucidiesPer100k'

df_gender = df.drop(['Country', 'Age', 'Year', 'CountryYear', 'GdpForYear', 'GdpPerCapital', 'Generation'], axis=1)

# group data based on Gender

gender_group = df_gender.groupby(['Gender'], as_index=False).sum()


# reset the SucidiesPer100k
gender_group['SuicidesPer100k'] = (gender_group['NumOfSuicides'] / gender_group['Population']) * 100000

gender_group.head()

In [ ]:
gender_visualization = visualization(Title= 'Gender & Suicides',xlabel=gender_group['Gender'], ylabel=gender_group['SuicidesPer100k'], xTitle='Gender')

# **Conclusion**

<a id=4> </a>

By comparing each useful columns(Year, Age, GdpForYear, Generation, Gender) to suicide, we have found out the features which affects people to suicide. There was one column that surprised me. In GdpForYear, if country has more than $1T GDP for year, it is likely to have high percentage for suicide. I guess people have more depression and Obsessive Compulsive Disorder. Higher GDP means the country is well developed and contain various technology. People needs to study and learn not only existence technology but also new skills. In addition, the gap between povery and wealth will be very serious problem. Rather than rich people, poor people will have less chance to succes. These problems will lead people to suicide.

# **References**
<a id=5> </a>
1. https://www.matplotlib.org/
2. https://docs.python.org/3/library/re.html
3. https://www.kaggle.com/kralmachine/data-visualization-of-suicide-rates
4. https://www.kaggle.com/kralmachine/data-visualization-of-suicide-rates****